# Column Generation


In [1]:
import time

from exputils.extent.cg import calculate_extent_CG
from exputils.state.random_ket import make_random_quantum_state

In [2]:
def CG_for_data(n: int):
    seed = 0
    kind = "pure"
    print(f"{n=}")
    psi = make_random_quantum_state(kind, n, seed)
    t0 = time.perf_counter()
    stabilizer_extent, extends, max_values, _ = calculate_extent_CG(
        n, psi, verbose=False
    )
    t1 = time.perf_counter()
    print(f"{stabilizer_extent=} {t1-t0=}")

    with open(f"../data/CG/{kind}_{n}_data.txt", mode="w") as f:
        for i in range(len(extends)):
            print(f"{extends[i]} {max_values[i]}", file=f)

    with open(f"../data/CG/{kind}_{n}_time.txt", mode="w") as f:
        print(t1 - t0, file=f)

## for the case $n<=8$

We assume the `MAX_VALUES_SIZE` be 10000.


In [3]:
for n in [4, 5, 6, 7, 8]:
    CG_for_data(n)

n=4
CG: n=4, method='mosek'
start: calculate dots
[k|progress|range]: 1 | 4.96000e+02/3.67200e+04 | [0.00646, 0.58428] | 9[ms]
[k|progress|range]: 2 | 4.97600e+03/3.67200e+04 | [0.00248, 0.66150] | 16[ms]
[k|progress|range]: 3 | 2.03360e+04/3.67200e+04 | [0.21602, 0.69257] | 20[ms]
[k|progress|range]: 4 | 3.67200e+04/3.67200e+04 | [0.21602, 0.69257] | 21[ms]
 calculation time : 21[ms]
iteration: 1 / 30, Amat.shape = (16, 10000)
start: solve SOCP
stabilizer_extent=2.543604607010636
start: calculate dual dots
[k|progress|range]: 1 | 4.96000e+02/3.67200e+04 | [nan, nan] | 14[ms]
[k|progress|range]: 2 | 4.97600e+03/3.67200e+04 | [0.97383, 1.00000] | 19[ms]
[k|progress|range]: 3 | 2.03360e+04/3.67200e+04 | [0.97100, 1.00000] | 22[ms]
[k|progress|range]: 4 | 3.67200e+04/3.67200e+04 | [0.97071, 1.00000] | 26[ms]
 calculation time : 26[ms]
# of violations(LB): 0
OPTIMAL!
stabilizer_extent=2.543604607010636 t1-t0=1.3786659575998783
n=5
CG: n=5, method='mosek'
start: calculate dots
[k|progress|r

## for the case $n=9$

We assume the `MAX_VALUES_SIZE` be 100000.


In [4]:
for n in [9]:
    CG_for_data(n)

n=9
CG: n=9, method='mosek'
start: calculate dots
iteration: 1 / 100, Amat.shape = (512, 100000)
start: solve SOCP
stabilizer_extent=17.544989853248886
start: calculate dual dots
# of violations(LB): 330
iteration: 2 / 100, Amat.shape = (512, 2013)
start: solve SOCP
stabilizer_extent=17.533160105624848
start: calculate dual dots
# of violations(LB): 794
iteration: 3 / 100, Amat.shape = (512, 2457)
start: solve SOCP
stabilizer_extent=17.52335905552666
start: calculate dual dots
# of violations(LB): 351
iteration: 4 / 100, Amat.shape = (512, 2028)
start: solve SOCP
stabilizer_extent=17.520037308629146
start: calculate dual dots
# of violations(LB): 226
iteration: 5 / 100, Amat.shape = (512, 1893)
start: solve SOCP
stabilizer_extent=17.51873272081433
start: calculate dual dots
# of violations(LB): 118
iteration: 6 / 100, Amat.shape = (512, 1785)
start: solve SOCP
stabilizer_extent=17.51824442062978
start: calculate dual dots
# of violations(LB): 92
iteration: 7 / 100, Amat.shape = (512, 1